In [1]:
from keras.models import Sequential
from keras.layers import GRU, Dense
import numpy as np
import random

In [2]:
import pandas as pd

df = pd.read_csv("the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv")

In [3]:
patient_id = list(df["Patient ID"])
recording_loc = list(df["Recording locations:"])
murmur = list(df["Murmur"])
murmur_loc = list(df["Murmur locations"])
systolic_murmur_timing = list(df["Systolic murmur timing"])
print(patient_id)
print(recording_loc)
print(murmur)
print(murmur_loc)
print(systolic_murmur_timing)

[2530, 9979, 9983, 13918, 14241, 14998, 23625, 24160, 29045, 29378, 31737, 33151, 36327, 38337, 39043, 39403, 39456, 40058, 40798, 40840, 43852, 44514, 45843, 46065, 46532, 46579, 46778, 47002, 49558, 49561, 49562, 49568, 49572, 49574, 49577, 49585, 49595, 49598, 49607, 49610, 49618, 49622, 49627, 49628, 49630, 49631, 49638, 49641, 49653, 49659, 49661, 49669, 49678, 49683, 49687, 49691, 49704, 49712, 49719, 49729, 49735, 49745, 49748, 49751, 49754, 49761, 49776, 49808, 49821, 49823, 49824, 49829, 49832, 49838, 49839, 49842, 49850, 49853, 49854, 49873, 49876, 49896, 49897, 49900, 49930, 49931, 49946, 49952, 49959, 49960, 49963, 49966, 49968, 49969, 49970, 49974, 49978, 49979, 49980, 49983, 49986, 49987, 49988, 49989, 49990, 49993, 49994, 49995, 49998, 49999, 50001, 50004, 50005, 50006, 50007, 50008, 50009, 50012, 50014, 50015, 50017, 50018, 50023, 50026, 50027, 50029, 50030, 50032, 50034, 50037, 50043, 50047, 50048, 50049, 50053, 50054, 50056, 50057, 50061, 50066, 50067, 50070, 50072, 5

In [4]:
recording_loc = [x.split("+") for x in recording_loc]

for i in range(len(murmur_loc)):
    if murmur_loc[i] is np.nan:
        murmur_loc[i] = []
    else:
        murmur_loc[i] = murmur_loc[i].split("+")

In [5]:
import scipy.io.wavfile as wav


X = []
y = []

for i in range(len(patient_id)):
    if murmur[i] == "Absent":
        for recloc in recording_loc[i]:
            try:
                samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + ".wav")
            except:
                samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + "_1.wav")
            initial = 0
            while (initial+400) <= len(full_recording):
                X.append(full_recording[initial:initial+400])
                y.append([1, 0, 0, 0, 0])
                initial += 200
    elif murmur[i] == "Present":
        for recloc in recording_loc[i]:
            if recloc in murmur_loc[i]:
                try:
                    samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + ".wav")
                except:
                    samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + "_1.wav")
                initial = 0
                if systolic_murmur_timing[i] == "Holosystolic":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 1, 0, 0, 0])
                        initial += 200
                elif systolic_murmur_timing[i] == "Early-systolic":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 1, 0, 0])
                        initial += 200
                elif systolic_murmur_timing[i] == "Mid-systolic":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 1, 0])
                        initial += 200
                elif systolic_murmur_timing[i] == "Late-systolic":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 0, 1])
                        initial += 200
                else:
                    print("Error on patient ID:", patient_id[i])
            else:
                try:
                    samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + ".wav")
                except:
                    samplerate, full_recording = wav.read("./the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/" + str(patient_id[i]) + "_" + str(recloc) + "_1.wav")
                initial = 0
                while (initial+400) <= len(full_recording):
                    X.append(full_recording[initial:initial+400])
                    y.append([1, 0, 0, 0, 0])
                    initial += 200

Error on patient ID: 85119
Error on patient ID: 85119


In [6]:
time_steps = 400
feature_length = 1
batch_size = 256

In [7]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.25)

In [8]:
X_train_final = []
y_train_final = []

for i in range(len(X_train)):
    if y_train[i] == [1, 0, 0, 0, 0]:
        #randnum = random.uniform(0, 1)
        randnum = random.random()
        if randnum >= 0.66:
            X_train_final.append(X_train[i])
            y_train_final.append(y_train[i])
    else:
        X_train_final.append(X_train[i])
        y_train_final.append(y_train[i])

In [9]:
print(len(X_train), len(y_train))
print(len(X_train_final), len(y_train_final))
print(len(X_test), len(y_test))

1027032 1027032
460331 460331
342344 342344


In [10]:
X_train_final = np.array(X_train_final)
y_train_final = np.array(y_train_final)

In [11]:
X_train_final = X_train_final.reshape((X_train_final.shape[0], X_train_final.shape[1], feature_length))

In [12]:
model = Sequential()
model.add(GRU(50, input_shape=(time_steps, feature_length)))
model.add(Dense(18, activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(5, activation="sigmoid"))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 50)                7950      
                                                                 
 dense (Dense)               (None, 18)                918       
                                                                 
 dense_1 (Dense)             (None, 128)               2432      
                                                                 
 dense_2 (Dense)             (None, 16)                2064      
                                                                 
 dense_3 (Dense)             (None, 5)                 85        
                                                                 
Total params: 13449 (52.54 KB)
Trainable params: 13449 (52.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [14]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [15]:
model.fit(X_train_final, y_train_final, epochs=3, batch_size=batch_size)

Epoch 1/3


1799/1799 [==============================] - 459s 254ms/step - loss: 0.1118 - accuracy: 0.6269
Epoch 2/3
1799/1799 [==============================] - 469s 261ms/step - loss: 0.1043 - accuracy: 0.6335
Epoch 3/3
1799/1799 [==============================] - 475s 264ms/step - loss: 0.1040 - accuracy: 0.6335


In [16]:
final_final_predicts = model.predict(X_test)
final_final_predicts.shape

10699/10699 [==============================] - 207s 19ms/step


(342344, 5)

In [17]:
X = np.array(X)
y = np.array(y)
score_full = model.evaluate(X, y, batch_size=batch_size, verbose=0)
print('Test loss:', score_full[0])
print('Accuracy:', score_full[1])

Test loss: 0.06454864889383316
Accuracy: 0.8355374932289124


In [18]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print('Test loss:', score[0])
print('Accuracy:', score[1])

Test loss: 0.0646577924489975
Accuracy: 0.8349584341049194


In [19]:
final_final_predicts

array([[0.72898793, 0.14144437, 0.11101988, 0.02363273, 0.00610171],
       [0.66272247, 0.1864404 , 0.10609135, 0.03050621, 0.00533607],
       [0.64003193, 0.20315707, 0.10525652, 0.03332314, 0.00512367],
       ...,
       [0.6782469 , 0.17556271, 0.10711722, 0.02879383, 0.00549502],
       [0.73241276, 0.13931067, 0.11155326, 0.02340248, 0.00616419],
       [0.6288122 , 0.21232495, 0.10496943, 0.0350056 , 0.00508214]],
      dtype=float32)

In [20]:
for i in range(len(final_final_predicts)):
    if max(final_final_predicts[i]) != final_final_predicts[i][0]:
        print(i, final_final_predicts[i])

In [21]:
full_predicts = model.predict(X)
full_predicts

42793/42793 [==============================] - 813s 19ms/step


array([[0.7445877 , 0.13175294, 0.11332737, 0.0224711 , 0.0063703 ],
       [0.70616823, 0.15604658, 0.1089135 , 0.02573524, 0.00576957],
       [0.6736739 , 0.17859562, 0.1066943 , 0.02919799, 0.00542801],
       ...,
       [0.47732556, 0.35302964, 0.09785189, 0.06353831, 0.00474196],
       [0.6160623 , 0.22269498, 0.1043358 , 0.03683105, 0.00502543],
       [0.6047267 , 0.23196076, 0.10399996, 0.03850988, 0.00495489]],
      dtype=float32)

In [22]:
for i in range(len(full_predicts)):
    if max(full_predicts[i]) != full_predicts[i][0]:
        print(i, full_predicts[i])

In [30]:
np.argmax(final_final_predicts, axis=1)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [32]:
#%% Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix, classification_report 

# Classification Report
model_report = classification_report(np.argmax(final_final_predicts, axis=1), np.argmax(y_test, axis=1))
print(model_report)

              precision    recall  f1-score   support

           0       1.00      0.83      0.91    342344
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.83    342344
   macro avg       0.20      0.17      0.18    342344
weighted avg       1.00      0.83      0.91    342344



C:\Users\savit\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\savit\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\savit\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_conf = confusion_matrix(np.argmax(y_test), axis=1), final_final_predicts)
print(model_conf)

In [ ]:
#%% ROC-AUC Curve

y_score = model.predict_proba(X_test)

from scipy import interp
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
# Learn to predict each class against the other


n_classes = 5 # number of class




# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Process of plotting roc-auc curve belonging to all classes.

from itertools import cycle

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-Class Systolic Murmur Timing LSTM')
plt.legend(loc="lower right")
plt.show()